<a href="https://colab.research.google.com/github/cryoTUD/ColabScale/blob/main/ColabScale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://gitlab.tudelft.nl/aj-lab/locscale/-/raw/master/doc/img/LocScale_logo.png" height="200" align="right" style="height:240px">

#```ColabScale```

Easy to use cryo-EM map sharpening using [```LocScale```](https://gitlab.tudelft.nl/aj-lab/locscale) and generation of feature-enhanced maps with [```LocScale-EMmerNet```](https://gitlab.tudelft.nl/aj-lab/emmernet).

For more details, see <a href="#Instructions">bottom</a> of the notebook and read our manuscripts.


In [ ]:
# @title Setup environment
%%capture
%%time
import os

if not os.path.isfile("CONDA_READY"):
   print("installing conda...")
   os.system("pip install -q condacolab")
   import condacolab
   condacolab.install()
   os.system("conda install -c conda-forge gfortran")
   os.system("conda create -n locscale python=3.8")
   os.system("touch CONDA_READY")

# downgrade CUDA version to 11.1 (for compatibility with tf model)
!wget https://developer.download.nvidia.com/compute/cuda/11.1.0/local_installers/cuda-repo-ubuntu2004-11-1-local_11.1.0-455.23.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-1-local_11.1.0-455.23.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-1-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-11-1
!export CUDA_PATH=/usr/local/cuda-11.1/

# install monomer library
!mkdir /usr/local/monomer_lib
!git clone https://github.com/MonomerLibrary/monomers.git /usr/local/monomer_lib
!export CLIBD_MON=/usr/local/monomer_lib/

# downgrade base python to Python3.8
!conda init
!conda install python=3.8 --no-pin
!pip install locscale
!pip install ipykernel

# include side packages from 3.8
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')



In [ ]:
#@markdown ### Input

from google.colab import files
import os

jobname = 'test' #@param {type:"string"}
job_type = "feature_enhance" # @param ["model-based", "model-free", "hybrid", "feature_enhance"]
#@markdown - __model-based__: ```LocScale``` sharpening using atomic model
#@markdown - __model-free__: ```LocScale``` sharpening without atomic model
#@markdown - __hybrid__: ```LocScale``` sharpening with partial atomic model
#@markdown - __feature_enhance__: Confidence-aware density modification with ```LocScale-EMmerNet```


use_model = False #@param {type:"boolean"}
if use_model == True:
  input_model_path = os.path.join(jobname,f"model")
  os.makedirs(input_model_path, exist_ok=True)
  print("Please select model file...")
  uploaded = files.upload()
  for model in uploaded.keys():
     if model.endswith('.pdb'):
        os.rename(model,os.path.join(input_model_path,model))
     else:
        print("Uploaded file is not a PDB file; please select correct file...")
        os.remove(model)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_model_path,model))
     use_model = True
elif use_model == False:
   input_model_path = None

#@markdown - `True` will request upload of atomic model as scaling reference. ADP refinement proceeds automaticallly.
#@markdown - `False` enables model-free scaling.

input_model_path = "/content/map.mrc"

use_half_maps = True #@param {type:"boolean"}
if use_half_maps == True:
  input_half_map_path = os.path.join(jobname,f"half_maps")
  os.makedirs(input_half_map_path, exist_ok=True)
  print("Please select half maps...")
  uploaded = files.upload()
  for map in uploaded.keys():
     if map.endswith('.mrc'):
        os.rename(map,os.path.join(input_half_map_path,map))
     else:
        print("Uploaded file is not a MRC file; please select correct file...")
        os.remove(map)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_half_map_path,map))
     use_half_maps = True
elif use_half_maps == False:
   input_half_map_path = None

#@markdown - Use of half maps is recommended.

#@markdown #### Other options

#model_resolution = None #@param {type:"string"}
symmetry = "C1" #@param {type:"string"}

In [ ]:
!locscale feature_enhance -np 8 -em map.mrc -ma mask.mrc -v -o test_fe_sym.mrc -gpus 0 -sym C4